In [1]:
display(dbutils.fs.ls("/mnt/mnt/tm-train"))

path,name,size
dbfs:/mnt/mnt/tm-train/billings_input.csv,billings_input.csv,4571297
dbfs:/mnt/mnt/tm-train/cars.csv,cars.csv,22663
dbfs:/mnt/mnt/tm-train/cereal.csv,cereal.csv,5157
dbfs:/mnt/mnt/tm-train/factbook.csv,factbook.csv,64856
dbfs:/mnt/mnt/tm-train/myzip.zip,myzip.zip,35002
dbfs:/mnt/mnt/tm-train/stock_data11122019.csv,stock_data11122019.csv,5123


In [2]:
import time
#time.localtime(1589793212000 )
time.strftime('%Y-%m-%d', time.localtime(1590083661000))

Out[63]: '52357-09-30'

In [3]:
from datetime import date

today = date.today()

# dd/mm/YY
d1 = today.strftime("%d/%m/%Y")
d1

Out[52]: '2020-05-25'

In [4]:
l=[]
hadoop = sc._jvm.org.apache.hadoop

fs = hadoop.fs.FileSystem
conf = hadoop.conf.Configuration() 
path = hadoop.fs.Path('/mnt/mnt/tm-train')

for f in fs.get(conf).listStatus(path):
  print(f.getModificationTime(),f.getPath())
  if today.strftime('%Y-%m-%d') == time.strftime('%Y-%m-%d', time.localtime(f.getModificationTime())):
    print('l')
    l.append(f.getPath())


1589793212000 dbfs:/mnt/mnt/tm-train/billings_input.csv
1590395747000 dbfs:/mnt/mnt/tm-train/cars.csv
1590395749000 dbfs:/mnt/mnt/tm-train/cereal.csv
1590395750000 dbfs:/mnt/mnt/tm-train/factbook.csv
1590083661000 dbfs:/mnt/mnt/tm-train/myzip.zip
1589794007000 dbfs:/mnt/mnt/tm-train/stock_data11122019.csv
Out[60]: []

In [5]:
import zipfile
from pyspark.sql.functions import monotonically_increasing_id,col
from pyspark.sql.types import StructType,StructField,StringType,IntegerType,FloatType
import os

In [6]:
df_source=spark.read.csv('dbfs:/mnt/mnt/tm-train/cereal.csv',schema=cerealschema,header=True,sep=';',mode='permissive',columnNameOfCorruptRecord='corrupt_record')
df_source.show()

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-326911826966019> in <module> 
 ----> 1 df_source = spark . read . csv ( 'dbfs:/mnt/mnt/tm-train/cereal.csv' , schema = cerealschema , header = True , sep = ';' , mode = 'permissive' , columnNameOfCorruptRecord = 'corrupt_record' ) 
 2 df_source . show ( ) 

 NameError : name 'cerealschema' is not defined

In [7]:
carschema=StructType([StructField('Car', StringType()),
        StructField('MPG', FloatType()),
        StructField('Cylinders', IntegerType()),
        StructField('Displacement', FloatType()),
        StructField('Horsepower', FloatType()),
        StructField('Weight', FloatType()),
        StructField('Acceleration', FloatType()),
        StructField('Model', IntegerType()),
        StructField('Origin', StringType())])

cerealschema=StructType([StructField('name', StringType()),
        StructField('mfr', StringType()),
        StructField('type', StringType()),
        StructField('calories', IntegerType()),
        StructField('protein', IntegerType()),
        StructField('fat', IntegerType()),
        StructField('sodium', IntegerType()),
        StructField('fiber', FloatType()),
        StructField('carbo', FloatType()),
        StructField('sugars', IntegerType()),
        StructField('potass', IntegerType()),
        StructField('vitamins', IntegerType()),
        StructField('shelf', IntegerType()),
        StructField('weight', FloatType()),
        StructField('cuos', FloatType()),
        StructField('rating', FloatType())])

In [8]:
df_source=spark.read.csv('dbfs:/mnt/mnt/tm-train/cereal.csv',schema=cerealschema,header=True,sep=';')
df_source=df_source.withColumn('Index',monotonically_increasing_id()).filter(col('Index') > 0).drop('Index')
df_carsource=spark.read.csv('dbfs:/mnt/mnt/tm-train/cars.csv',schema=carschema,header=True,sep=';')
df_carsource=df_carsource.withColumn('Index',monotonically_increasing_id()).filter(col('Index') > 0).drop('Index')
df_carsource.show()

+--------------------+----+---------+------------+----------+------+------------+-----+------+
 Car| MPG|Cylinders|Displacement|Horsepower|Weight|Acceleration|Model|Origin|
+--------------------+----+---------+------------+----------+------+------------+-----+------+
Chevrolet Chevell...|18.0| 8| 307.0| 130.0|3504.0| 12.0| 70| US|
 Buick Skylark 320|15.0| 8| 350.0| 165.0|3693.0| 11.5| 70| US|
 Plymouth Satellite|18.0| 8| 318.0| 150.0|3436.0| 11.0| 70| US|
 AMC Rebel SST|16.0| 8| 304.0| 150.0|3433.0| 12.0| 70| US|
 Ford Torino|17.0| 8| 302.0| 140.0|3449.0| 10.5| 70| US|
 Ford Galaxie 500|15.0| 8| 429.0| 198.0|4341.0| 10.0| 70| US|
 Chevrolet Impala|14.0| 8| 454.0| 220.0|4354.0| 9.0| 70| US|
 Plymouth Fury iii|14.0| 8| 440.0| 215.0|4312.0| 8.5| 70| US|
 Pontiac Catalina|14.0| 8| 455.0| 225.0|4425.0| 10.0| 70| US|
 AMC Ambassador DPL|15.0| 8| 390.0| 190.0|3850.0| 8.5| 70| US|
Citroen DS-21 Pallas| 0.0| 4| 133.0| 115.0|3090.0| 17.5| 70|Europe|
Chevrolet Chevell...| 0.0| 8| 350.0| 165.0|4142.0| 11.5| 70| US|
 Ford Torino (sw)| 0.0| 8| 351.0| 153.0|4034.0| 11.0| 70| US|
Plymouth Satellit...| 0.0| 8| 383.0| 175.0|4166.0| 10.5| 70| US|
 AMC Rebel SST (sw)| 0.0| 8| 360.0| 175.0|3850.0| 11.0| 70| US|
 Dodge Challenger SE|15.0| 8| 383.0| 170.0|3563.0| 10.0| 70| US|
 Plymouth 'Cuda 340|14.0| 8| 340.0| 160.0|3609.0| 8.0| 70| US|
Ford Mustang Boss...| 0.0| 8| 302.0| 140.0|3353.0| 8.0| 70| US|
Chevrolet Monte C...|15.0| 8| 400.0| 150.0|3761.0| 9.5| 70| US|
Buick Estate Wago...|14.0| 8| 455.0| 225.0|3086.0| 10.0| 70| US|
+--------------------+----+---------+------------+----------+------+------------+-----+------+
only showing top 20 rows

In [9]:
df_source.withColumn('Index',monotonically_increasing_id()).filter(col('Index') <10).drop('Index').createOrReplaceTempView("cereal")
df_carsource.withColumn('Index',monotonically_increasing_id()).filter(col('Index') <10).drop('Index').createOrReplaceTempView("cars")
results = spark.sql("SELECT count(*) FROM cars")
results.show()

+--------+
count(1)|
+--------+
 10|
+--------+

In [10]:
results = spark.sql("SELECT count(*) FROM default.cereal")
results.show()

+--------+
count(1)|
+--------+
 402|
+--------+

In [11]:
def process(li):
  for i in li:
    if 'cars' in i:
      print('first i',i)
      fileschema=carschema
      jkey='Car'
      dtab='default.cars'
    if 'cereal' in i:
      print('second i',i)
      fileschema=cerealschema
      jkey='name'
      dtab='default.cereal'

    # if condition for checking only extracted

    if 'cars' in i or 'cereal'  in i:
      print(i)
      df_source=spark.read.csv(i,schema=fileschema,header=True,sep=';')
      #display(df)
      df_source=df_source.withColumn('Index',monotonically_increasing_id()).filter(col('Index') > 0).drop('Index')
      df_tab=spark.table(dtab)
      
      #removing duplicate records
      
      df_source=df_source.dropDuplicates()
      #print(df_source.count())
      newdf=df_source.join(df_tab, on=[jkey], how='left_anti')
      print(newdf.count(),i,dtab)
      newdf.write.mode("append").saveAsTable(dtab)
      

In [12]:
newdf.show()

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-326911826966018> in <module> 
 ----> 1 newdf . show ( ) 

 NameError : name 'newdf' is not defined

In [13]:

if os.stat('/dbfs/mnt/mnt/tm-train/myzip.zip').st_size != 0:
  zf = zipfile.ZipFile('/dbfs/mnt/mnt/tm-train/myzip.zip')
  zf.extractall('/dbfs/mnt/mnt/tm-train')
  rdd=sc.wholeTextFiles('dbfs:/mnt/mnt/tm-train')
  li=rdd.map(lambda x:x[0]).collect()
  process(li)

first i dbfs:/mnt/mnt/tm-train/cars.csv
dbfs:/mnt/mnt/tm-train/cars.csv
19 dbfs:/mnt/mnt/tm-train/cars.csv default.cars
second i dbfs:/mnt/mnt/tm-train/cereal.csv
dbfs:/mnt/mnt/tm-train/cereal.csv
10 dbfs:/mnt/mnt/tm-train/cereal.csv default.cereal

In [14]:
results = spark.sql("SELECT count(*) FROM default.cars")
results.show()

+--------+
count(1)|
+--------+
 2322|
+--------+